In [18]:
import yfinance as yf
import datetime
import pandas as pd
gld = yf.Ticker('GLD')
uso = yf.Ticker('USO')
uup = yf.Ticker('UUP')

#### Volatility arbitrage Strategy

* When the ratio of ~30 day to ~90 day vol is less than 1 we are in a short vol position
* When the ratio of ~30 day to ~90 day vol is greater than 1 we are in a long vol position




### What does this mean?

## Long Vol Positions 
#### When ~30 / ~90 > 1 it suggests market participants are buying front-month protection and expecting volatility to increase. 
 
 ### We want to be long straddles (buy at the money calls and puts) 
<img src="long_straddle.png" width="800" height="400">

### A less profitable and less risky position is a long strangle (buy slightly out of the money calls and puts) 
<img src="long_strangle.png" width="800" height="400">
    
    
## Short VoL Positions
#### When ~30 / ~90 < 1 it suggests market participants are less worried about front-month risks and risk further out on the term structure is greater.

### We want to be short straddles (Sell at the money calls and puts) 


### We want to be short strangles (Sell slightly out of the money calls and puts) 

In [2]:
datetime.date.today()

datetime.date(2020, 5, 18)

### Let's Look at GLD Options Dates

* GLD options are priced weekly for the front month. 
* We'll use the 6th week contract and the contract two months of that to get our roughly 30/90 day ratio

In [3]:
gld.options

('2020-05-21',
 '2020-05-28',
 '2020-06-04',
 '2020-06-11',
 '2020-06-18',
 '2020-06-25',
 '2020-06-29',
 '2020-07-16',
 '2020-09-17',
 '2020-09-29',
 '2020-11-19',
 '2020-12-17',
 '2020-12-30',
 '2021-01-14',
 '2021-03-18',
 '2021-03-30',
 '2021-06-17',
 '2021-09-16',
 '2021-11-18',
 '2021-12-16',
 '2022-01-20')

#### We need to know the current price in order to determine at the money strike prices for calls and puts

In [13]:
current_price = (gld.info['ask'] + gld.info['bid'])/2


### Let's start by calculating the front month at the money call implied vol
* We get the contract and then look for strikes within 1 dollar of the current price
* We then get the take the mean implied volatility from the options at those strikes
* I am going for the mean because it is easy and does not really affect the overall price too much
* We see the difference between the two implied vols is about .005 or half a percent

In [162]:
gld.option_chain(gld.options[5]).calls.head(5)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,GLD200619C00070000,2020-04-28 14:37:34,70.0,89.85,90.4,90.6,0.0,0.0,52.0,274,0.992188,True,REGULAR,USD
1,GLD200619C00075000,2020-04-30 17:02:58,75.0,83.95,85.4,85.6,0.0,0.0,2.0,29,0.914063,True,REGULAR,USD
2,GLD200619C00080000,2020-05-06 13:44:00,80.0,79.60,80.4,80.6,0.0,0.0,5.0,41,0.843752,True,REGULAR,USD
3,GLD200619C00085000,2020-04-01 13:30:07,85.0,63.42,0.0,0.0,0.0,0.0,2.0,21,0.000010,True,REGULAR,USD
4,GLD200619C00090000,2020-03-18 19:26:44,90.0,51.40,68.1,68.7,0.0,0.0,11.0,14,0.000010,True,REGULAR,USD


In [144]:
one_month_gld_calls.loc[
                        (one_month_gld_calls['strike'] <= (current_price + 1)) & 
                        (one_month_gld_calls['strike'] >= (current_price - 1))]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
62,GLD200619C00160000,2020-05-08 19:59:56,160.0,5.19,5.15,5.25,-0.81,-13.499999,1294.0,38985,0.232307,True,REGULAR,USD
63,GLD200619C00161000,2020-05-08 19:52:47,161.0,4.74,4.70,4.90,-0.71,-13.027524,2543.0,28007,0.238411,False,REGULAR,USD


In [142]:
one_month_gld_calls = gld.option_chain(gld.options[5]).calls

one_month_call_vol = (one_month_gld_calls.loc[
                        (one_month_gld_calls['strike'] <= (current_price + 1)) & 
                        (one_month_gld_calls['strike'] >= (current_price - 1))]
                        ['impliedVolatility'].mean()
           )
one_month_call_vol

0.23535920898437498

### We do the same for front month  puts 


In [169]:
one_month_gld_puts = gld.option_chain(gld.options[5]).puts

In [170]:
one_month_gld_puts.head(5)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,GLD200619P00070000,2020-04-30 18:54:55,70.0,0.01,0.0,0.01,0.0,0.0,11.0,705,0.765627,False,REGULAR,USD
1,GLD200619P00075000,2020-04-28 17:30:33,75.0,0.01,0.0,0.01,0.0,0.0,40.0,100,0.703128,False,REGULAR,USD
2,GLD200619P00080000,2020-05-01 19:54:53,80.0,0.01,0.0,0.02,0.0,0.0,5.0,410,0.687503,False,REGULAR,USD
3,GLD200619P00085000,2020-04-30 18:13:19,85.0,0.01,0.0,0.01,0.0,0.0,10.0,268,0.593754,False,REGULAR,USD
4,GLD200619P00090000,2020-05-01 18:47:56,90.0,0.01,0.0,0.02,0.0,0.0,10.0,9353,0.578129,False,REGULAR,USD


In [171]:
(one_month_gld_puts.loc[
                        (one_month_gld_puts['strike'] <= (current_price + 1)) & 
                        (one_month_gld_puts['strike'] >= (current_price - 1))])
                        

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
66,GLD200619P00160000,2020-05-08 19:59:03,160.0,4.66,4.6,4.70,0.08,1.746723,848.0,10099,0.226448,False,REGULAR,USD
67,GLD200619P00161000,2020-05-08 19:50:01,161.0,5.28,5.1,5.35,0.23,4.554456,3901.0,1589,0.232552,True,REGULAR,USD


In [172]:


one_month_put_vol = (one_month_gld_puts.loc[
                        (one_month_gld_puts['strike'] <= (current_price + 1)) & 
                        (one_month_gld_puts['strike'] >= (current_price - 1))]
                        ['impliedVolatility'].mean()
           )
one_month_put_vol

0.22949989257812498

In [173]:
one_month_implied_vol = (one_month_call_vol + one_month_put_vol)/2

In [174]:
one_month_implied_vol

0.23242955078125

### We need to do the same for 90 day calls 

* We can see the contract expires in September which is 90 days out from the previous contract

In [175]:
gld.option_chain(gld.options[8]).calls.head(5)

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,GLD200918C00074000,2020-03-19 16:54:15,74.0,65.45,84.25,85.15,0.0,0.0,NaN,0,0.00001,True,REGULAR,USD
1,GLD200918C00077000,2020-03-20 19:58:00,77.0,74.90,81.25,82.15,0.0,0.0,1.0,0,0.00001,True,REGULAR,USD
2,GLD200918C00078000,2020-03-02 05:04:26,78.0,77.95,69.80,71.05,0.0,0.0,NaN,37,0.00001,True,REGULAR,USD
3,GLD200918C00079000,2020-03-27 08:16:08,79.0,77.25,73.80,74.85,0.0,0.0,1.0,34,0.00001,True,REGULAR,USD
4,GLD200918C00080000,2020-03-19 17:44:27,80.0,59.25,78.30,79.00,0.0,0.0,105.0,0,0.00001,True,REGULAR,USD


In [177]:
three_month_gld_calls = gld.option_chain(gld.options[8]).calls

three_month_call_vol = (three_month_gld_calls.loc[
                        (three_month_gld_calls['strike'] <= (current_price + 1)) & 
                        (three_month_gld_calls['strike'] >= (current_price - 1))]
                        ['impliedVolatility'].mean()
           )
three_month_call_vol

0.25997138961791993

In [178]:
three_month_gld_puts = gld.option_chain(gld.options[8]).puts

three_month_put_vol = (three_month_gld_puts.loc[
                        (three_month_gld_puts['strike'] <= (current_price + 1)) & 
                        (three_month_gld_puts['strike'] >= (current_price - 1))]
                        ['impliedVolatility'].mean()
           )
three_month_put_vol

0.2528456063842774

In [180]:
three_month_implied_vol = (three_month_call_vol + three_month_put_vol)/2
three_month_implied_vol

0.25640849800109866

In [182]:
vol_ratio = one_month_implied_vol / three_month_implied_vol
vol_ratio

0.9064814645115782

In [15]:
## Do it all at once 
from numba import jit
import yfinance as yf

def get_data(ticker):
    ticker = yf.Ticker(ticker)
    
    return ticker

def compute_term_iv(ticker, term):
    
    series = get_data(ticker)
    
    current_price = (gld.info['ask'] + gld.info['bid'])/2

    one_month_calls = series.option_chain(series.options[term]).calls
    
    one_month_implied_vol_calls = (one_month_calls.loc[
                                        (one_month_calls['strike'] <= (current_price + 1)) & 
                                        (one_month_calls['strike'] >= (current_price - 1))]
                                        ['impliedVolatility'].mean()
                                    )
    
    one_month_puts = series.option_chain(series.options[term]).puts
    
    one_month_implied_vol_puts = (one_month_puts.loc[
                                        (one_month_puts['strike'] <= (current_price + 1)) & 
                                        (one_month_puts['strike'] >= (current_price - 1))]
                                      ['impliedVolatility'].mean()
                                )
    
    return (one_month_implied_vol_puts + one_month_implied_vol_calls)/2

def compute_vol_term_ratio(ticker, term1, term2):
    
    term1_vol = compute_term_iv(ticker, term1)
    
    term2_vol = compute_term_iv(ticker, term2)  
    
    return (term1_vol/term2_vol)

In [26]:
compute_term_iv('USO', 0)

0.872071591796875

In [30]:
compute_term_iv('GLD', 8)

0.22897346961975096

In [33]:
compute_vol_term_ratio('USO', 4, 7)

1.0965371333609157

In [28]:
uso.options

('2020-05-21',
 '2020-05-28',
 '2020-06-04',
 '2020-06-11',
 '2020-06-18',
 '2020-06-25',
 '2020-07-16',
 '2020-10-15',
 '2021-01-14',
 '2022-01-20',
 '2022-12-15')

### Conclusion

* We see that we are currently at 0.906 for a vol ratio
* This implies declining volatility relative to future months
* We would want to take a short straddle/strangle position to profit from this
* Based on our ratio, we know when to take enter/exit positions and therefore we know when to take profit/loss


#### Further Discussion and Questions 

* How should we work this? Straddles or Strangles?
* Do we buy only one month or spread our position across multiple medium term future month maturities
* One example for a long position would be to one straddle at front month, 2 at next month out, 3 at 3 months and 3 at 5 months
* I think for simplicity, right now, it is easier to buy 1 at 2 months out 1 at 3 months out. 
 
#### Next Steps

* Set up the infra to pull daily data and save to S3
* Set up infra to execute trades using IB
* Set up some kind of front-end to do other anlaysis

